In [5]:
%store -r TOKENIZER_FILE
import pickle
with open(TOKENIZER_FILE, "rb") as fr:
    tokenizer = pickle.load(fr)

# Describe Model

## compare two model's embedding weight

In [2]:
import os
import numpy as np

MODEL_FILE = os.path.join(SOURCE_DIR, MODEL_FN)
MODEL_FILE_1 = os.path.join(SOURCE_DIR, MODEL_FN_1)

import tensorflow as tf
from sklearn.metrics import classification_report
model = tf.keras.models.load_model(MODEL_FILE)
model_1 = tf.keras.models.load_model(MODEL_FILE_1)

amount_same = np.sum(model.layers[1].weights[0] == model_1.layers[1].weights[0])
amount_total = (lambda x: x[0]*x[1])(model_1.layers[1].weights[0].shape)
print(
    f"totally there are {amount_total} elements in the weight, "
    f"the amount same elements are: {amount_same}"
)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-01-14 11:23:57.598171: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-14 11:23:57.598859: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


totally there are 19549200 elements in the weight, the amount same elements are: 2019730


### how many words are still with zero in weights

In [3]:
total_words = model.layers[1].weights[0].shape[0]
dim = model.layers[1].weights[0].shape[1]
not_trained_words = np.sum(np.sum(model.layers[1].weights[0]==0, axis=1)==dim)

print(
    f"there are {total_words} in the matrix, "
    f"finally still are {not_trained_words} words with all zero wights"
)


there are 65164 in the matrix, finally still are 4962 words with all zero wights


In [8]:
pass
# np.argwhere((np.sum(model.layers[1].weights[0]==0, axis=1)==dim)==True)
for idx in np.argwhere((np.sum(model.layers[1].weights[0]==0, axis=1)==dim)==True):
    term = tokenizer.index_word[idx]
    df_no_exist_word[df_no_exist_word.term==term]

/Users/vincentcheng/miniforge3/envs/tm_nlp/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydev_bundle/_pydev_imports_tipper.py:205: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  obj = getattr(obj_to_complete, d)
/Users/vincentcheng/miniforge3/envs/tm_nlp/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydev_bundle/_pydev_imports_tipper.py:205: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  obj = getattr(obj_to_complete, d)
/Users/vincentcheng/miniforge3/envs/tm_nlp/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydev_bundle/_pydev_imports_tipper.py:205: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  obj = getattr(obj_to_complete, d)


In [7]:
%store -r NO_EXIST_WORDS_FILE
import pandas as pd
df_no_exist_word = pd.read_csv(NO_EXIST_WORDS_FILE)

df_tokenizer_map = pd.DataFrame.from_dict(tokenizer.index_word, orient='index')


# 模型相关的统计分析

## 训练集和测试集的cls分布

In [3]:
def do_cls_stat(np_class):
    np_class = np_class.argmax(axis=1)+1
    np_class.sort()
    unique_idx = np.split(np.unique(np_class, return_index=True)[1], 45)
    unique_idx = [int(item) for item in unique_idx]
    unique_idx.append(np_class.shape[0])
    cls_stat = [unique_idx[i + 1] - unique_idx[i]  for i in range(45)]
    return cls_stat


In [4]:
cls_stat_train = do_cls_stat(np_class_train)
if sum(cls_stat_train) != np_class_train.shape[0]:
    raise ValueError("class stat error")

cls_stat_test = do_cls_stat(np_class_test)
if sum(cls_stat_test) != np_class_test.shape[0]:
    raise ValueError("class stat error")

In [ ]:
cls_stat_test

## 训练集和测试集在准确率（based on cls）

In [20]:
import pickle

with open(TOKENIZED_GS_PICKLE_FILE, "rb") as fr:
    tokenized_gs = pickle.load(fr)

with open(TOKENIZED_CLASS_PICKLE_FILE, "rb") as fr:
    tokenized_class = pickle.load(fr)

import tensorflow as tf
from sklearn.metrics import classification_report
model = tf.keras.models.load_model(MODEL_FILE)

predictions = model.predict(tokenized_gs)
report = classification_report(
    tokenized_class.argmax(axis=1),
	predictions.argmax(axis=1)
)
print(report)

 10/344 [..............................] - ETA: 4s

2022-12-30 10:54:48.216684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


344/344 [==============================] - 4s 12ms/step
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       203
           1       0.98      1.00      0.99        60
           2       1.00      0.97      0.98       398
           3       1.00      0.98      0.99        42
           4       1.00      1.00      1.00       250
           5       1.00      0.97      0.98       189
           6       0.98      0.98      0.98       345
           7       0.97      0.97      0.97        78
           8       0.97      1.00      0.98      1404
           9       0.99      0.99      0.99       201
          10       0.99      0.96      0.98       336
          11       0.99      0.99      0.99       222
          12       0.85      0.96      0.90        24
          13       0.97      0.97      0.97        96
          14       0.88      1.00      0.94        22
          15       0.98      0.99      0.98       583
          16       0.99  

/Users/vincentcheng/miniforge3/envs/tm_nlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vincentcheng/miniforge3/envs/tm_nlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vincentcheng/miniforge3/envs/tm_nlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           0       0.84      0.79      0.82      3138
           1       0.78      0.83      0.80       796
           2       0.89      0.93      0.91      3807
           3       0.86      0.74      0.80       489
           4       0.89      0.88      0.88      3453
           5       0.81      0.86      0.84      2382
           6       0.80      0.82      0.81      4928
           7       0.80      0.78      0.79      1190
           8       0.87      0.92      0.89     17958
           9       0.87      0.81      0.84      2929
          10       0.82      0.76      0.79      2874
          11       0.78      0.80      0.79      2310
          12       0.84      0.82      0.83       271
          13       0.77      0.91      0.83       987
          14       0.84      0.84      0.84       289
          15       0.92      0.91      0.91      7473
          16       0.70      0.75      0.72      1412
          17       0.87    